### This notebook contains the code to generate the preprocessed data for the "Pump it Up: Data Mining the Water Table" competition.

We have the training data and corresponding values, as well as test data to eventually generate the submission for.

We drop duplicate columns, impute missing values, engineer new features, and apply one-hot encoding.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [ ]:
X = pd.read_csv('../data/training_values.csv')
X_test = pd.read_csv('../data/test_values.csv')
y = pd.read_csv('../data/training_labels.csv')
X.iloc[:,:20].head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd


In [4]:
X.iloc[:,20:].head()

,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [5]:
# Drop some useless columns: id (but keep in test data for submission), wpt_name (just the name of the waterpoint), 
# num_private (not stated what it means, most values are 0 anyway), recorder_by (irrelevant and all values are the same),
# payment_type (duplicate of payment), quantity_group (duplicate of quantity)
print(X['num_private'].value_counts()/len(X))
print(X['recorded_by'].value_counts())
#print(X['extraction_type'].unique(), '\n', X['extraction_type_group'].unique(), '\n', X['extraction_type_class'].unique())  # these do get more general
print(X['payment_type'].unique(), X['payment'].unique())
#print(X['water_quality'].unique(), '\n', X['quality_group'].unique())  # also does get more general
print(X['quantity'].unique(), '\n', X['quantity_group'].unique())
#print(X['source'].unique(), '\n', X['source_type'].unique(), '\n', X['source_class'].unique())  # also does get more general
#print(X['waterpoint_type'].unique(), '\n', X['waterpoint_type_group'].unique())  # also does get more general

# Note we can also try to take out the more general features as there is stills some redundancy
X.drop(columns=['id', 'wpt_name', 'num_private', 'recorded_by', 'payment_type', 'quantity_group'], inplace=True)
X_test.drop(columns=['wpt_name', 'num_private', 'recorded_by', 'payment_type', 'quantity_group'], inplace=True)

num_private
0       0.987256
6       0.001364
1       0.001229
5       0.000774
8       0.000774
          ...   
42      0.000017
23      0.000017
136     0.000017
698     0.000017
1402    0.000017
Name: count, Length: 65, dtype: float64
recorded_by
GeoData Consultants Ltd    59400
Name: count, dtype: int64
['annually' 'never pay' 'per bucket' 'unknown' 'on failure' 'other'
 'monthly'] ['pay annually' 'never pay' 'pay per bucket' 'unknown'
 'pay when scheme fails' 'other' 'pay monthly']
['enough' 'insufficient' 'dry' 'seasonal' 'unknown'] 
 ['enough' 'insufficient' 'dry' 'seasonal' 'unknown']


In [6]:
# Check how many values there are for: basin, subvillage, region, region_code, district_code, lga, ward
# to see if any are the same and must be dropped
print(X['basin'].nunique())
print(X['subvillage'].nunique())
print(X['region'].nunique())
print(X['region_code'].nunique())
print(X['district_code'].nunique())
print(X['lga'].nunique())
print(X['ward'].nunique())
# So there are far too many subvillage, we can't one-hot encode them all, so drop them
# I don't know how there are more region names than codes, but we'll keep the codes as there are more of those
# We also have a lot (too many for one-hot encoding) of lga and ward values, so drop those, but then keep district code and basin
X.drop(columns=['subvillage', 'region', 'lga', 'ward'], inplace=True)
X_test.drop(columns=['subvillage', 'region', 'lga', 'ward'], inplace=True)

9
19287
21
27
20
125
2092


In [7]:
# Change data_recorded into just the year, then combine that with construction year to get age of the waterpoint
X['date_recorded'] = pd.to_datetime(X['date_recorded'])
X_test['date_recorded'] = pd.to_datetime(X_test['date_recorded'])
X['construction_year'] = X['construction_year'].replace(0, np.nan)  # data contains 0, which must mean unknown
X_test['construction_year'] = X_test['construction_year'].replace(0, np.nan)
X['age'] = X['date_recorded'].dt.year - X['construction_year']
X_test['age'] = X_test['date_recorded'].dt.year - X_test['construction_year']
X.drop(columns=['date_recorded', 'construction_year'], inplace=True)
X_test.drop(columns=['date_recorded', 'construction_year'], inplace=True)
X.head()  # for the missing values we impute the mean later

,amount_tsh,funder,gps_height,installer,longitude,latitude,basin,region_code,district_code,population,...,payment,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type,waterpoint_type_group,age
0,6000.0,Roman,1390,Roman,34.938093,-9.856322,Lake Nyasa,11,5,109,...,pay annually,soft,good,enough,spring,spring,groundwater,communal standpipe,communal standpipe,12.0
1,0.0,Grumeti,1399,GRUMETI,34.698766,-2.147466,Lake Victoria,20,2,280,...,never pay,soft,good,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,3.0
2,25.0,Lottery Club,686,World vision,37.460664,-3.821329,Pangani,21,4,250,...,pay per bucket,soft,good,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,4.0
3,0.0,Unicef,263,UNICEF,38.486161,-11.155298,Ruvuma / Southern Coast,90,63,58,...,never pay,soft,good,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,27.0
4,0.0,Action In A,0,Artisan,31.130847,-1.825359,Lake Victoria,18,1,0,...,never pay,soft,good,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,NaN


In [8]:
# For y we also drop the id column
y.drop(columns='id', inplace=True)
# Then the status is an ordinal variable, with non functional being the worst, needing repair being better, and functional being best,
# so we can just encode that as 0, 1, 2
y['status_group'] = y['status_group'].map({'non functional': 0, 'functional needs repair': 1, 'functional': 2})
y.head()

,status_group
0,2
1,2
2,2
3,0
4,2


In [9]:
# Check for missing values (none in y) (same columns for X_test)
X.isna().sum()[ X.isna().sum() > 0 ]

funder                3637
installer             3655
public_meeting        3334
scheme_management     3878
scheme_name          28810
permit                3056
age                  20709
dtype: int64

In [10]:
# Before imputing these, check if there are not so many unique values that we can't one-hot encode them
# This applies to non-numeric columns, so all except age
missing_values = X.isna().sum()[ X.isna().sum() > 0 ].index
missing_values = missing_values.drop('age')

cols_to_drop = []
for col in missing_values:
    print(col, X[col].nunique())
    if X[col].nunique() > 50:  # arbitrary threshold
        cols_to_drop.append(col)

X.drop(columns=cols_to_drop, inplace=True)
X_test.drop(columns=cols_to_drop, inplace=True)
X.head()

funder 1896
installer 2145
public_meeting 2
scheme_management 11
scheme_name 2695
permit 2


,amount_tsh,gps_height,longitude,latitude,basin,region_code,district_code,population,public_meeting,scheme_management,...,payment,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type,waterpoint_type_group,age
0,6000.0,1390,34.938093,-9.856322,Lake Nyasa,11,5,109,True,VWC,...,pay annually,soft,good,enough,spring,spring,groundwater,communal standpipe,communal standpipe,12.0
1,0.0,1399,34.698766,-2.147466,Lake Victoria,20,2,280,NaN,Other,...,never pay,soft,good,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,3.0
2,25.0,686,37.460664,-3.821329,Pangani,21,4,250,True,VWC,...,pay per bucket,soft,good,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,4.0
3,0.0,263,38.486161,-11.155298,Ruvuma / Southern Coast,90,63,58,True,VWC,...,never pay,soft,good,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,27.0
4,0.0,0,31.130847,-1.825359,Lake Victoria,18,1,0,True,NaN,...,never pay,soft,good,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,NaN


In [11]:
# Check if any other object columns also have too many unique values for one-hot encoding and need to be dropped
# (better do this before imputing missing values)
object_cols = X.select_dtypes(include='object').columns
more_cols_to_drop = []
for col in object_cols:
    print(col, X[col].nunique())
    if X[col].nunique() > 50:  # arbitrary threshold
        more_cols_to_drop.append(col)
# No columns to drop

basin 9
public_meeting 2
scheme_management 11
permit 2
extraction_type 18
extraction_type_group 13
extraction_type_class 7
management 12
management_group 5
payment 7
water_quality 8
quality_group 6
quantity 5
source 10
source_type 7
source_class 3
waterpoint_type 7
waterpoint_type_group 6


In [12]:
# Then before imputation, split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.isna().sum()[ X_train.isna().sum() > 0 ]

public_meeting        2689
scheme_management     3103
permit                2439
age                  16503
dtype: int64

In [13]:
# Only 'age' is numerical, impute mean
imputer = SimpleImputer(strategy='mean')
X_train['age'] = imputer.fit_transform(X_train[['age']])
X_val['age'] = imputer.transform(X_val[['age']])
X_test['age'] = imputer.transform(X_test[['age']])

In [14]:
# For public_meeting, scheme_management, permit, we add the 'Unknown' category for missing values
missing_values = X_train.isna().sum()[ X_train.isna().sum() > 0 ].index

for col in missing_values:
    X_train[col] = X_train[col].fillna('Unknown')
    X_val[col] = X_val[col].fillna('Unknown')
    X_test[col] = X_test[col].fillna('Unknown')

X_train.iloc[:,:14].head()

,amount_tsh,gps_height,longitude,latitude,basin,region_code,district_code,population,public_meeting,scheme_management,permit,extraction_type,extraction_type_group,extraction_type_class
3607,50.0,2092,35.426020,-4.227446,Internal,21,1,160,True,Water Board,True,gravity,gravity,gravity
50870,0.0,0,35.510074,-5.724555,Internal,1,6,0,True,VWC,True,india mark ii,india mark ii,handpump
20413,0.0,0,32.499866,-9.081222,Lake Rukwa,12,6,0,True,VWC,False,other,other,other
52806,0.0,0,34.060484,-8.830208,Rufiji,12,7,0,True,VWC,True,gravity,gravity,gravity
50091,300.0,1023,37.032690,-6.040787,Wami / Ruvu,5,1,120,True,VWC,True,other,other,other


In [15]:
X_train.iloc[:,14:].head()

,management,management_group,payment,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type,waterpoint_type_group,age
3607,water board,user-group,pay per bucket,soft,good,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,15.000000
50870,vwc,user-group,never pay,soft,good,enough,shallow well,shallow well,groundwater,hand pump,hand pump,15.348615
20413,vwc,user-group,never pay,soft,good,enough,shallow well,shallow well,groundwater,other,other,15.348615
52806,vwc,user-group,pay monthly,soft,good,insufficient,river,river/lake,surface,communal standpipe,communal standpipe,15.348615
50091,vwc,user-group,pay when scheme fails,salty,salty,enough,shallow well,shallow well,groundwater,other,other,14.000000


In [16]:
# Next deal with the remaining object columns; they are all nominal
# Note that the region_code and district_code are numbers, but they are actually categorical, so we should one-hot encode them as well
object_cols = X_train.select_dtypes(include='object').columns
object_cols = object_cols.append(pd.Index(['region_code', 'district_code']))
print(object_cols)

X_train = pd.get_dummies(X_train, columns=object_cols, dtype=int)
X_val = pd.get_dummies(X_val, columns=object_cols, dtype=int)
X_test = pd.get_dummies(X_test, columns=object_cols, dtype=int)
X_train.head()

Index(['basin', 'public_meeting', 'scheme_management', 'permit',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'water_quality',
       'quality_group', 'quantity', 'source', 'source_type', 'source_class',
       'waterpoint_type', 'waterpoint_type_group', 'region_code',
       'district_code'],
      dtype='object')


,amount_tsh,gps_height,longitude,latitude,population,age,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,...,district_code_23,district_code_30,district_code_33,district_code_43,district_code_53,district_code_60,district_code_62,district_code_63,district_code_67,district_code_80
3607,50.0,2092,35.426020,-4.227446,160,15.000000,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50870,0.0,0,35.510074,-5.724555,0,15.348615,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20413,0.0,0,32.499866,-9.081222,0,15.348615,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
52806,0.0,0,34.060484,-8.830208,0,15.348615,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50091,300.0,1023,37.032690,-6.040787,120,14.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
X_val.head()

,amount_tsh,gps_height,longitude,latitude,population,age,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,...,district_code_23,district_code_30,district_code_33,district_code_43,district_code_53,district_code_60,district_code_62,district_code_63,district_code_67,district_code_80
2980,0.0,0,31.985658,-3.596360,0,15.348615,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5246,0.0,0,32.832815,-4.944937,0,15.348615,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
22659,10.0,1675,35.488289,-4.242048,148,5.000000,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39888,0.0,0,33.140828,-9.059386,0,15.348615,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
13361,50.0,1109,34.217077,-4.430529,235,2.000000,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# There seem to be 2 missing columns in the validation data
print(set(X_train.columns) - set(X_val.columns))  # appears in X_train but not in X_val
# These are both one-hot encoded columns, so we can just add them with all 0s
X_val['region_code_40'] = 0
X_val['extraction_type_other - mkulima/shinyanga'] = 0
X_val = X_val[X_train.columns]  # put columns in same order
X_val.head()

{'region_code_40', 'extraction_type_other - mkulima/shinyanga'}


,amount_tsh,gps_height,longitude,latitude,population,age,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,...,district_code_23,district_code_30,district_code_33,district_code_43,district_code_53,district_code_60,district_code_62,district_code_63,district_code_67,district_code_80
2980,0.0,0,31.985658,-3.596360,0,15.348615,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5246,0.0,0,32.832815,-4.944937,0,15.348615,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
22659,10.0,1675,35.488289,-4.242048,148,5.000000,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39888,0.0,0,33.140828,-9.059386,0,15.348615,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
13361,50.0,1109,34.217077,-4.430529,235,2.000000,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
X_test.head()

,id,amount_tsh,gps_height,longitude,latitude,population,age,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,...,district_code_23,district_code_30,district_code_33,district_code_43,district_code_53,district_code_60,district_code_62,district_code_63,district_code_67,district_code_80
0,50785,0.0,1996,35.290799,-4.059696,321,1.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,51630,0.0,1569,36.656709,-3.309214,300,13.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,17168,0.0,1567,34.767863,-5.004344,500,3.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,45559,0.0,267,38.058046,-9.418672,250,26.0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,49871,500.0,1260,35.006123,-10.950412,60,13.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# There seem to be 2 missing columns in X_test too, as it should have 1 more than X_train (id) but instead has 1 less
print(set(X_train.columns) - set(X_test.columns))  # appears in X_train but not in X_test
# These are both one-hot encoded columns, so we can just add them with all 0s
X_test['region_code_40'] = 0
X_test['extraction_type_other - mkulima/shinyanga'] = 0

# Put columns in same order, and keep id
id = X_test['id']
X_test = X_test[X_train.columns]
X_test['id'] = id
X_test.head()

{'region_code_40', 'extraction_type_other - mkulima/shinyanga'}


,amount_tsh,gps_height,longitude,latitude,population,age,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,...,district_code_30,district_code_33,district_code_43,district_code_53,district_code_60,district_code_62,district_code_63,district_code_67,district_code_80,id
0,0.0,1996,35.290799,-4.059696,321,1.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,50785
1,0.0,1569,36.656709,-3.309214,300,13.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,51630
2,0.0,1567,34.767863,-5.004344,500,3.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,17168
3,0.0,267,38.058046,-9.418672,250,26.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,45559
4,500.0,1260,35.006123,-10.950412,60,13.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,49871


In [ ]:
# Save the data
X_train.to_csv('../prep_data/X_train.csv', index=False)
X_val.to_csv('../prep_data/X_val.csv', index=False)
X_test.to_csv('../prep_data/X_test.csv', index=False)
y_train.to_csv('../prep_data/y_train.csv', index=False)
y_val.to_csv('../prep_data/y_val.csv', index=False)